# Social and Informational Influences

This section explores how social context and information sources shape the adoption of electric vehicles (EVs) in France. Survey data was analyzed to understand the relationships between:

- **Peer influence**  
  (Number of EV users in the respondent’s social circle, discussions about EVs, encouragement or discouragement from peers, Perceived opinion of peers, Impact of peer exchanges)
- **Clarity and trust in information sources**  
  (Perceived clarity and reliability of information about EVs, Main information sources)
- **Brand communication**  
  (Brands perceived as offering clear and well-communicated EV options)
- **Sociodemographic effects**  
  (Influence of age, professional status, and geographic location on adoption intent)

## Setup

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from scipy.stats import chi2_contingency
import re
from scipy.stats import kruskal
import pandas as pd
from scipy.stats import f_oneway
from scipy.stats import spearmanr
from IPython.display import display

In [ ]:
def extract_after_newline_parentheses(col):
    match = re.search(r'\n\s*\((.*?)\)', col)
    if match:
        return match.group(1)
    match = re.search(r'\(([^()]*(?:\([^)]*\)[^()]*)*)\)\s*$', col)
    if match:
        return match.group(1)
    return None

def extract_before_newline_parentheses(col):
    match = re.search(r'^(.*?)\s*\n', col)
    if match:
        return match.group(1)
    match = re.search(r'^(.*?)\s*\(', col)
    if match:
        return match.group(1)
    return col
def get_labels_and_title(cols):
    labels = [extract_after_newline_parentheses(col) for col in cols]
    title = extract_before_newline_parentheses(cols[0])
    return labels, title



In [ ]:
df = pd.read_csv(r'c:\Users\Mango\Downloads\FinalEvAdoptionInFranceResults.csv')
for col in df.columns:
    values = df[col].dropna().astype(str)
    split_vals = []
    for val in values:
        split_vals.extend([v.strip() for v in val.split(',') if v.strip()])
    unique_vals = pd.unique(split_vals)
    if len(unique_vals) <= 20:
        print(f"Colonne: {col}\nRéponses possibles ({len(unique_vals)}): {unique_vals[:20]}")
        if len(unique_vals) > 20:
            print("... (troncature à 20 valeurs)")
        print("-" * 60)
print(list(df.columns))

C:\Users\Mango\AppData\Local\Temp\ipykernel_43280\507991816.py:7: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_vals = pd.unique(split_vals)
C:\Users\Mango\AppData\Local\Temp\ipykernel_43280\507991816.py:7: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_vals = pd.unique(split_vals)
C:\Users\Mango\AppData\Local\Temp\ipykernel_43280\507991816.py:7: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_vals = pd.unique(split_vals)
C:\Users\Mango\AppData\Local\Temp\ipykernel_43280\507991816.py:7: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_vals = pd.unique(

Colonne: Combien êtes-vous dans votre foyer ?
Réponses possibles (10): ['4' '2' '5' '1' '3' '6' '9' '7' '8' '11']
------------------------------------------------------------
Colonne: Habitez-vous dans :
Réponses possibles (4): ['Une ville moyenne' 'Une grande ville' 'Une zone périurbaine'
 'Une zone rurale']
------------------------------------------------------------
Colonne: Avez-vous accès à une place de parking personnelle ?
Réponses possibles (4): ['Oui'
 "avec borne de recharge ou possibilité d'en installer une (à domicile ou en résidence par exemple)"
 'Non' "sans borne de recharge ou possibilité d'en installer une"]
------------------------------------------------------------
Colonne: Combien de véhicules personnels utilisez-vous dans votre foyer ?
Réponses possibles (7): ['2' '0' '1' '3' '4' '7' '5']
------------------------------------------------------------
Colonne: Pourquoi n'avez-vous pas de véhicule ?

Réponses possibles (7): ["Je n'ai pas de permis de conduire"
 'Je vi

KeyboardInterrupt: 

## **Peer influence**  
  (Number of EV users in the respondent’s social circle, discussions about EVs, encouragement or discouragement from peers, Perceived opinion of peers, Impact of peer exchanges)

In [ ]:
peer_cols = [
    "Combien de personnes dans votre entourage utilisent un véhicule électrique ?\n",
    "Vous diriez que [Vous avez déjà échangé sur le sujet des véhicules électriques avec votre entourage]",
    "Vous diriez que [l'avis de votre entourage vous à encourager dans l'adoption du véhicule électrique]",
    "Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique]",
    "Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet]"
]
for col in peer_cols:
    print(f"\n{col}")
    print(df[col].value_counts(dropna=False))
    if df[col].dtype == object:
        # Recoder Oui/Non si besoin
        df[col + "_bin"] = df[col].map({"Oui": 1, "Non": 0, "Vaguement": 0.5, "Je ne sais pas": np.nan})
        group = df.groupby(col + "_bin")["intention_adopter"].mean()
        print("Intention d'adopter selon ce facteur :\n", group)


for col in peer_cols[1:]:  # Sauf la première qui est quantitative
    group1 = df[df[col + "_bin"] == 1]["intention_adopter"].dropna()
    group0 = df[df[col + "_bin"] == 0]["intention_adopter"].dropna()
    if len(group1) > 1 and len(group0) > 1:
        tstat, pval = ttest_ind(group1, group0, equal_var=False)
        print(f"{col} : t = {tstat:.2f}, p = {pval:.4f}")
        if pval < 0.05:
            print("✅ Différence significative entre les groupes.")
        else:
            print("❌ Pas de différence significative entre les groupes.")

try:
    entourage_nb = pd.to_numeric(df[peer_cols[0]], errors='coerce')
    corr = entourage_nb.corr(df['intention_adopter'])
    print(f"Corrélation entre nombre d'utilisateurs VE dans l'entourage et intention d'adopter : r = {corr:.2f}")
except Exception as e:
    print("Erreur de calcul de corrélation :", e)


Combien de personnes dans votre entourage utilisent un véhicule électrique ?

Combien de personnes dans votre entourage utilisent un véhicule électrique ?\n
Peu (1 à 3)             121
Aucune                   64
Plusieurs (3 à 5)        30
Beaucoup (plus de 5)     17
Name: count, dtype: int64
Intention d'adopter selon ce facteur :
 Series([], Name: intention_adopter, dtype: float64)

Vous diriez que [Vous avez déjà échangé sur le sujet des véhicules électriques avec votre entourage]
Vous diriez que [Vous avez déjà échangé sur le sujet des véhicules électriques avec votre entourage]
Totalement d'accord        110
Plutôt d'accord             82
Plutôt en désaccord         28
Totalement en désaccord     12
Name: count, dtype: int64
Intention d'adopter selon ce facteur :
 Series([], Name: intention_adopter, dtype: float64)

Vous diriez que [l'avis de votre entourage vous à encourager dans l'adoption du véhicule électrique]
Vous diriez que [l'avis de votre entourage vous à encourager dans

In [ ]:
for col in ["Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique]", "Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet]"]:
    # Mappe les 4 options d'accord/désaccord sur une échelle 0-3
    map4 = {
        "Totalement en désaccord": 0,
        "Plutôt en désaccord": 1,
        "Plutôt d'accord": 2,
        "Totalement d'accord": 3
    }
    df[col + "_bin"] = df[col].map(map4)
    group1 = df[df[col + "_bin"] == 3]["intention_adopter"].dropna()
    group0 = df[df[col + "_bin"] == 0]["intention_adopter"].dropna()
    print(f"{col} - n1: {len(group1)}, n0: {len(group0)}, unique1: {group1.unique()}, unique0: {group0.unique()}")
    print(f"Var group1: {group1.var()}, Var group0: {group0.var()}")
    if len(group1) > 1 and len(group0) > 1:
        tstat, pval = ttest_ind(group1, group0, equal_var=False)
        print(f"{col} : t = {tstat:.2f}, p = {pval:.4f}")
    else:
        print(f"{col} : Pas assez de données pour le t-test.")

Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique] - n1: 15, n0: 9, unique1: [1. 0.], unique0: [0. 1.]
Var group1: 0.17142857142857146, Var group0: 0.19444444444444445
Vous diriez que [L'avis de votre entourage est positif envers le véhicule électrique] : t = 3.18, p = 0.0058
Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet] - n1: 10, n0: 22, unique1: [1. 0.], unique0: [0. 1.]
Var group1: 0.1, Var group0: 0.08658008658008656
Vous diriez que [Échanger avec eux vous a rassuré ou motivé sur ce sujet] : t = 6.85, p = 0.0000


## **Clarity and trust in information sources**  
  (Perceived clarity and reliability of information about EVs, Main information sources)

In [ ]:
clarity_col = "Ces sources vous paraissent-elles claires et fiables ?\n"
df["clarity_bin"] = df[clarity_col].map({"Oui, j’ai facilement trouvé ce que je cherchais": 1, "Pas toujours, il faut croiser les infos et c'était plutôt complexe de comprendre les offres et véhicules disponible sur le marché": 0})
group = df.groupby("clarity_bin")["intention_adopter"].mean()
print("Intention d'adopter selon la clarté perçue des sources :\n", group)

Intention d'adopter selon la clarté perçue des sources :
 clarity_bin
0.0    0.629213
1.0    0.692308
Name: intention_adopter, dtype: float64


In [ ]:
info_col = "Où avez-vous cherché (ou chercheriez-vous) des informations sur les véhicules électriques ?\n"
sources = df[info_col].dropna().str.split(',').explode().str.strip()
source_counts = sources.value_counts()
print("Sources d'information les plus utilisées :\n", source_counts.head(10))

Sources d'information les plus utilisées :
 Où avez-vous cherché (ou chercheriez-vous) des informations sur les véhicules électriques ?\n
Entourage (amis                               125
collègues…)                                   125
Sites des constructeurs / Concessionnaires    111
Médias généralistes / spécialisés             105
Comparateurs de véhicules                      74
Forums / avis en ligne                         64
Réseaux sociaux                                42
Salons ou événements                           23
Non                                             2
YouTube / Influenceurs automobile               2
Name: count, dtype: int64


## **Brand communication**  
  (Brands perceived as offering clear and well-communicated EV options)

In [ ]:
brand_col = "Quelles marques vous semblent proposer une offre électrique claire et bien communiquée ?\n"
brands = df[brand_col].dropna().str.split(',').explode().str.strip()
brand_counts = brands.value_counts()
print("Marques les plus citées :\n", brand_counts.head(10))

Marques les plus citées :
 Quelles marques vous semblent proposer une offre électrique claire et bien communiquée ?\n
Tesla         97
Renault       76
Peugeot       47
BYD           41
Aucune        32
BMW           25
Kia           25
Volkswagen    24
Hyundai       24
Mercedes      20
Name: count, dtype: int64


## **Sociodemographic effects**  
  (Influence of age, professional status, and geographic location on adoption intent)

In [ ]:
from scipy.stats import ttest_ind, f_oneway

# H2 : ANOVA ou t-test selon l'âge, la CSP, la zone géographique
for var in ["Dans quelle tranche d’âge vous situez-vous ?\n", "Habitez-vous dans :", "Quelle est votre situation professionnelle actuelle ?\n"]:
    print(f"\nEffet de {var} sur l'intention d'adopter :")
    groups = [g["intention_adopter"].dropna() for _, g in df.groupby(var) if g["intention_adopter"].notna().sum() > 1]
    if len(groups) >= 2:
        fval, pval = f_oneway(*groups)
        print(f"ANOVA: F = {fval:.2f}, p = {pval:.4f}")
        # Analyse automatique
        if pval < 0.05:
            print("✅ Il existe une différence significative de l'intention d'adopter selon cette variable.")
            means = [g.mean() for g in groups]
            print(f"  Moyennes par groupe : {means}")
        else:
            print("❌ Pas de différence significative de l'intention d'adopter selon cette variable.")
    else:
        print("Pas assez de groupes pour faire une ANOVA valide.")

# H3 : t-test clarté perçue vs intention
group1 = df[df["clarity_bin"] == 1]["intention_adopter"].dropna()
group0 = df[df["clarity_bin"] == 0]["intention_adopter"].dropna()
tstat, pval = ttest_ind(group1, group0, equal_var=False)
print(f"H3 - Clarté perçue : t = {tstat:.2f}, p = {pval:.4f}")
# Analyse automatique
if pval < 0.05:
    print("✅ L'intention d'adopter est significativement plus élevée chez ceux qui trouvent les sources claires et fiables.")
    print(f"  Moyenne (sources claires) : {group1.mean():.2f}")
    print(f"  Moyenne (sources peu claires) : {group0.mean():.2f}")
else:
    print("❌ Pas de différence significative d'intention d'adopter selon la clarté perçue des sources.")


Effet de Dans quelle tranche d’âge vous situez-vous ?
 sur l'intention d'adopter :
ANOVA: F = 0.80, p = 0.5719
❌ Pas de différence significative de l'intention d'adopter selon cette variable.

Effet de Habitez-vous dans : sur l'intention d'adopter :
ANOVA: F = 2.77, p = 0.0435
✅ Il existe une différence significative de l'intention d'adopter selon cette variable.
  Moyennes par groupe : [np.float64(0.6451612903225806), np.float64(0.5111111111111111), np.float64(0.41379310344827586), np.float64(0.7619047619047619)]

Effet de Quelle est votre situation professionnelle actuelle ?
 sur l'intention d'adopter :
ANOVA: F = 0.44, p = 0.7821
❌ Pas de différence significative de l'intention d'adopter selon cette variable.
H3 - Clarté perçue : t = 0.60, p = 0.5538
❌ Pas de différence significative d'intention d'adopter selon la clarté perçue des sources.


## **multivariate (cross) analyses** 